# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key
geoapify_key

'277b1436c72347f2bd2fdc6336093d77'

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Prince Rupert,CA,54.3161,-130.3201,44.29,71,100,29.93,light rain
1,1,Rongelap,MH,11.1635,166.8941,81.68,78,100,20.76,light rain
2,2,Margaret River,AU,-33.9500,115.0667,57.07,57,0,14.90,clear sky
3,3,Susuman,RU,62.7833,148.1667,1.98,97,100,1.99,overcast clouds
4,4,Saipan,MP,15.1355,145.7010,81.30,83,40,13.80,scattered clouds


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
# #%%capture --no-display

# Configure the map plot
coordinate_df = city_data_df[["Lat","Lng","City","Country","Humidity","Max Temp","Wind Speed"]]
# Display the map
map_plot = coordinate_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM")
map_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
coordinate_df = coordinate_df[(coordinate_df["Max Temp"]<90) & (coordinate_df["Humidity"]<50)]

# Drop any rows with null values
coordinate_df  = coordinate_df.dropna(how='any',axis=0)

# Display sample data
coordinate_df.head()

,Lat,Lng,City,Country,Humidity,Max Temp,Wind Speed
6,-9.5333,-77.5333,Huaraz,PE,49,67.60,4.92
31,27.0377,14.4283,Sabha,LY,40,60.85,15.79
49,42.0970,-79.2353,Jamestown,US,49,56.79,16.11
112,-15.4067,28.2871,Lusaka,ZM,24,84.92,7.43
115,13.5167,35.7667,Doka,SD,19,81.93,7.34


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= coordinate_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
len(hotel_df)

62

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
limit = 50

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row["Lng"]
    latitude =  row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias =    f"proximity:{longitude},{latitude}"
    params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key,
     "categories":"accommodation.hotel"}
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
   
    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)
    
# print(base_url + params)
    
#     # Convert the API response to JSON format
    name_address = name_address.json()
   
        
#     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
         hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

 # Display sample data
hotel_df.head()

Starting hotel search
Huaraz - nearest hotel: Andes Camp
Sabha - nearest hotel: فندق الكاظم-سبها
Jamestown - nearest hotel: DoubleTree Jamestown
Lusaka - nearest hotel: Lusaka Grand Hotel
Doka - nearest hotel: No hotel found
Kashgar - nearest hotel: 深航国际酒店
Faya-Largeau - nearest hotel: No hotel found
Senanga - nearest hotel: Senanga Safari Lodge
Illapel - nearest hotel: Domingo Ortiz de Rosas
Medina - nearest hotel: The Oberoi Madina
Jiayuguan City - nearest hotel: 嘉峪关宾馆
Kumul - nearest hotel: Хами
Chhor - nearest hotel: No hotel found
Ha'il - nearest hotel: فندق كلاسيك
Preobrazheniye - nearest hotel: No hotel found
Phalodi - nearest hotel: LAL NIWAS
Tamanghasset - nearest hotel: فندق الأمان
Carnarvon - nearest hotel: No hotel found
Al Jawf - nearest hotel: No hotel found
Sinaloa de Leyva - nearest hotel: No hotel found
Calvinia - nearest hotel: No hotel found
Picota - nearest hotel: No hotel found
Geraldton - nearest hotel: Ocean Centre Hotel
Porbandar - nearest hotel: Toran Tourist B

,City,Country,Lat,Lng,Humidity,Hotel Name
6,Huaraz,PE,-9.5333,-77.5333,49,Andes Camp
31,Sabha,LY,27.0377,14.4283,40,فندق الكاظم-سبها
49,Jamestown,US,42.0970,-79.2353,49,DoubleTree Jamestown
112,Lusaka,ZM,-15.4067,28.2871,24,Lusaka Grand Hotel
115,Doka,SD,13.5167,35.7667,19,No hotel found


In [46]:
hotel_df_Hotelfound = hotel_df[hotel_df["Hotel Name"]!="No hotel found"]
len(hotel_df_Hotelfound) 

40

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Configure the map plot (color = "city",)
map_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True,  tiles = "OSM")


# Display the map
map_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]